In [1]:
#import sys
#sys.path.append('/dss/dsshome1/0F/di93quv/Systems_biomedicine/acdc/')
from ACDC.random_walk_classifier import * 
from ACDC.cell_type_annotation import * 

In [15]:
#!pip install pandas numpy scikit-learn scipy seaborn matplotlib phenograph

In [10]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix
import phenograph
from sklearn.model_selection import StratifiedKFold
import time

In [3]:
channels = ['CD45','CD45RA', 'CD19', 'CD11b', 'CD4', 'CD8', 'CD34',
           'CD20', 'CD33', 'CD123', 'CD38', 'CD90', 'CD3']

# Load the data
path = '/dss/dsshome1/0F/di93quv/Systems_biomedicine/acdc/data/BMMC_benchmark/'
df = pd.read_csv(path + 'BMMC_benchmark.csv.gz', sep=',', header=0, compression='gzip')
df = df[df.cell_type != 'NotGated']


table = pd.read_csv(path + 'BMMC_table.csv', sep=',', header=0, index_col=0)
table = table.fillna(0)

cts, channels = get_label(table)

X0= np.arcsinh((df[channels].values - 1.0)/5.0)

In [4]:
# Define helper dictionaries
idx2ct = [key for idx, key in enumerate(table.index)]
idx2ct.append('unknown')

ct2idx = {key: idx for idx, key in enumerate(table.index)}
ct2idx['unknown'] = len(table.index)
        
ct_score = np.abs(table.to_numpy()).sum(axis=1)

## compute manual gated label
y0 = np.zeros(df.cell_type.shape)

for i, ct in enumerate(df.cell_type):
    if ct in ct2idx:
        y0[i] = ct2idx[ct]
    else:
        y0[i] = ct2idx['unknown']

In [9]:
n_neighbor = 10
thres = 0.5

In [11]:
# Initialize variables
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
result = []
score_final = []
process_time = []

c = 0
for tr, te in skf.split(X0, y0):
    print('%02d th batch' % c)
    if c == 1:
        break  # Stop after first batch for testing
    c += 1
    
    # Prepare data
    X = X0[tr, :]
    y_true = y0[tr]

    mk_model = compute_marker_model(pd.DataFrame(X, columns=channels), table, 0.0)

    # Compute posterior probabilities
    tic = time.perf_counter()
    score = get_score_mat(X, [], table, [], mk_model)
    score = np.concatenate([score, 1.0 - score.max(axis=1)[:, np.newaxis]], axis=1)

    # Get indices
    ct_index = get_unique_index(X, score, table, thres)

    # Baseline - classify events
    y_pred_index = np.argmax(score, axis=1)
    toc = time.perf_counter()
    time0 = toc - tic

    # Run ACDC
    tic = time.perf_counter()
    res_c = get_landmarks(X, score, ct_index, idx2ct, phenograph, thres)

    landmark_mat, landmark_label = output_feature_matrix(res_c, [idx2ct[i] for i in range(len(idx2ct))])
    landmark_label = np.array(landmark_label)

    lp, y_pred = rm_classify(X, landmark_mat, landmark_label, n_neighbor)
    toc = time.perf_counter()
    time1 = toc - tic

    # Run phenograph classification
    tic = time.perf_counter()
    res = phenograph.cluster(X, k=30, directed=False, prune=False, min_cluster_size=10, jaccard=True,
                              primary_metric='euclidean', n_jobs=-1, q_tol=1e-3)
    toc = time.perf_counter()
    time2 = toc - tic

    tic = time.perf_counter()
    y_pred_oracle = np.zeros_like(y_true)
    for i in range(max(res[0]) + 1):
        ic, nc = Counter(y_true[res[0] == i]).most_common(1)[0]
        y_pred_oracle[res[0] == i] = ic
    toc = time.perf_counter()
    time3 = toc - tic

    # Record accuracy scores
    score_final.append([
        accuracy_score(y_true, [ct2idx[c] for c in y_pred]),
        accuracy_score(y_true, y_pred_index),
        accuracy_score(y_true, y_pred_oracle)
    ])

    # Record times
    process_time.append((time0, time1, time2, time3))

    # Save results
    result.append((y_true, y_pred, y_pred_index, y_pred_oracle))
    pickle.dump(result, open('processed_file/BMMC/event_classification_BMMC.p', 'wb'))

# Output results
print("Scores:", score_final)
print("Processing times:", process_time)

00 th batch
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 4.3213701248168945 seconds
Jaccard graph constructed in 2.732866048812866 seconds
Wrote graph to binary file in 0.26862287521362305 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.646365
After 2 runs, maximum modularity is Q = 0.647876
After 6 runs, maximum modularity is Q = 0.651186
After 21 runs, maximum modularity is Q = 0.653661
Louvain completed 41 runs in 17.12009310722351 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 25.289024829864502 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 1.751307725906372 seconds
Jaccard graph constructed in 1.7989606857299805 seconds
Wrote graph to binary file in 0.1814587116241455 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.59936
After 6 runs, maximum modularity is Q = 0.6

FileNotFoundError: [Errno 2] No such file or directory: 'processed_file/BMMC/event_classification_BMMC.p'

In [12]:
np.mean(score_final, axis = 0) # score of ACDC, score-based classification, phenograph classification

array([0.92181599, 0.80006728, 0.94232151])

In [13]:
score_final

[[0.9218159854427573, 0.8000672813737634, 0.9423215132192608]]

In [14]:
np.mean(process_time, axis = 0)

array([2.73794864e-01, 2.09678506e+02, 1.22295478e+02, 4.06807610e-02])